In [1]:
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from tqdm import tqdm
from collections import defaultdict
from itertools import combinations
import random
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
data_dir = r"C:\Users\ishan\OneDrive\Desktop\face recog\newf"


c:\Users\ishan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Using device: cuda


In [3]:

# Load pretrained FaceNet model from facenet-pytorch
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Transformation to apply to each face image
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),  # FaceNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1,1]
])

embeddings = []
labels = []



In [4]:

# Loop through data directory
for person in os.listdir(data_dir):
    person_dir = os.path.join(data_dir, person)
    if not os.path.isdir(person_dir):
        continue
    for file in tqdm(os.listdir(person_dir), desc=f"Processing {person}"):
        img_path = os.path.join(person_dir, file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        # Convert BGR (OpenCV) to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Apply transformations
        face_tensor = transform(img_rgb).unsqueeze(0).to(device)  # Add batch dim and send to GPU
        
        # Get embedding from model (no gradients needed)
        with torch.no_grad():
            embedding = model(face_tensor).cpu().numpy()[0]  # Move to CPU, get numpy array
        
        embeddings.append(embedding)
        labels.append(person)

print(f"Extracted {len(embeddings)} embeddings.")


Processing Aaron_Peirsol:   0%|          | 0/4 [00:00<?, ?it/s]

Processing Zoran_Djindjic: 100%|██████████| 5/5 [00:00<00:00, 18.13it/s]

Extracted 5376 embeddings.


In [5]:
embeddings_t = [torch.tensor(e, dtype=torch.float32) for e in embeddings]

label_to_embs = defaultdict(list)
for emb, label in zip(embeddings_t, labels):
    label_to_embs[label].append(emb)

X = []
y = []

# Positive pairs (same person)
for label, embs in label_to_embs.items():
    if len(embs) < 2:
        continue
    for emb1, emb2 in combinations(embs, 2):
        pair = torch.cat([emb1, emb2])
        X.append(pair)
        y.append(torch.tensor(1, dtype=torch.float32))

# Negative pairs (different people)
all_labels = list(label_to_embs.keys())
for _ in range(len(y)):  # generate as many negatives as positives
    label1, label2 = random.sample(all_labels, 2)
    emb1 = random.choice(label_to_embs[label1])
    emb2 = random.choice(label_to_embs[label2])
    pair = torch.cat([emb1, emb2])
    X.append(pair)
    y.append(torch.tensor(0, dtype=torch.float32))

# Stack all tensors into one tensor
X = torch.stack(X)  # shape: (num_pairs, 2*embedding_dim)
y = torch.stack(y)  # shape: (num_pairs,)

print(f"Generated {len(X)} pairs.")
print("X shape:", X.shape)
print("y shape:", y.shape)

Generated 19350 pairs.
X shape: torch.Size([19350, 1024])
y shape: torch.Size([19350])


In [6]:

# Assuming X, y are torch tensors (from previous step)
# Move tensors to CPU for train_test_split, since sklearn expects numpy/cpu tensors
X_np = X.cpu().numpy()
y_np = y.cpu().numpy()

# Train/test split (using sklearn)
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

# Convert back to torch tensors
X_train = torch.tensor(X_train_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.float32)
X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_test = torch.tensor(y_test_np, dtype=torch.float32)

# Dataset and DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [7]:
class MLP(nn.Module):
    def __init__(self, input_dim=1024):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

model = MLP(input_dim=1024).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

epochs = 10

In [8]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).unsqueeze(1).float()

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_x.size(0)

        preds = (outputs > 0.5).float()
        correct_train += (preds == batch_y).sum().item()
        total_train += batch_y.size(0)

    epoch_loss = running_loss / total_train
    train_acc = correct_train / total_train
    print(f"Epoch {epoch+1}/{epochs} — Loss: {epoch_loss:.4f}, Accuracy: {train_acc:.4f}")


Epoch 1/10 — Loss: 0.6830, Accuracy: 0.5843
Epoch 2/10 — Loss: 0.5413, Accuracy: 0.7862
Epoch 3/10 — Loss: 0.3844, Accuracy: 0.8552
Epoch 4/10 — Loss: 0.3229, Accuracy: 0.8726
Epoch 5/10 — Loss: 0.2898, Accuracy: 0.8859
Epoch 6/10 — Loss: 0.2706, Accuracy: 0.8927
Epoch 7/10 — Loss: 0.2559, Accuracy: 0.8979
Epoch 8/10 — Loss: 0.2413, Accuracy: 0.9036
Epoch 9/10 — Loss: 0.2269, Accuracy: 0.9130
Epoch 10/10 — Loss: 0.2200, Accuracy: 0.9143


In [9]:
model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).unsqueeze(1).float()

        outputs = model(batch_x)
        preds = (outputs > 0.5).float()

        correct_test += (preds == batch_y).sum().item()
        total_test += batch_y.size(0)

test_acc = correct_test / total_test
print(f"\nTest Accuracy after training: {test_acc:.4f}")


Test Accuracy after training: 0.8809


In [10]:
np.save("embeddings.npy", np.array(embeddings))  # shape: (N, 512)
np.save("labels.npy", np.array(labels))    

In [11]:
torch.save(model.state_dict(), "face_match_model.pth")

In [12]:
torch.save(model, "face_match_full_model.pth")

In [15]:
def get_embedding(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Cannot read image: {img_path}")

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)

    img_tensor = transform(img_pil).unsqueeze(0).to(device)  # shape (1, 3, 160, 160)

    with torch.no_grad():
        embedding = model(img_tensor)  # shape: (1, 512)
    
    return embedding.squeeze(0).cpu().numpy()  # convert to NumPy (512,)

In [17]:
from PIL import Image

In [18]:
img1_path = r"C:\Users\ishan\OneDrive\Pictures\Camera Roll\WIN_20250525_11_28_48_Pro.jpg"
img2_path = r"C:\Users\ishan\OneDrive\Pictures\Camera Roll\WIN_20250601_18_19_16_Pro.jpg"

# Get embeddings
emb1 = get_embedding(img1_path)
emb2 = get_embedding(img2_path)

# Concatenate embeddings
pair_input = np.concatenate([emb1, emb2])  # shape: (1024,)

# Convert to torch tensor
pair_tensor = torch.tensor(pair_input, dtype=torch.float32).unsqueeze(0).to(device)

# Get prediction
with torch.no_grad():
    output = model(pair_tensor).item()

print(f"Similarity Score: {output:.4f}")
if output > 0.5:
    print("→ These faces are predicted to be the same person.")
else:
    print("→ These faces are predicted to be different people.")


TypeError: pic should be Tensor or ndarray. Got <class 'PIL.Image.Image'>.